In [2]:
pip install numpy pandas matplotlib


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import sys
import os 
sys.path.append("..")
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp


from src.gating import alpha_h , alpha_m, alpha_n , beta_h , beta_m, beta_n
from src.hh_model import hh_derivatives


ModuleNotFoundError: No module named 'currents'